# XGBoost

Objective function = Loss function $+$ Regularization Term = $L(\theta)+\Omega(\theta)$ 


The training loss measures how predictive our model is w.r.t. to the training data. 


For $\textbf{Regression}$, the common choice is $\textit{mean squared error}$, which is given by \
$L(\theta)=\sum_{i}(y_{i}-\hat{y}_{i})^2$ 

For $\textbf{Classification}$, we use $\textbf{Logistic Loss}$, given by \
$L(\theta)=\sum_{i}[y_i*ln(1+e^{-\hat{y}_i})+(1-y_i)*ln(1+e^{\hat{y}_i})]$

The regularization term, $\Omega(\theta)$, controls the complexity of the model, which helps us to avoid overfitting.

Mathematically, $\hat{y}_i$ is given by \
$\begin{alignat}{1}\hat{y}_i=\sum_{k=1}^{K}f_{k}(x_i), f_k \in \mathcal{F}\end{alignat}$ \
where, $K$ is the number of trees, $f_k$ is a function in functional space $\mathcal{F}$, and $\mathcal{F}$ is the set of all possible CARTs.

$\implies \hat{y}^{(t)}_i = \hat{y}^{(t-1)}_i+f_t(x_i)$

The objective function for the model, at any time step $t$, is given by \
$\begin{alignat}{1}\textit{obj}^{(t)}&=\sum_{i=1}^nl(y_i,\hat{y}_i^t)+\sum_{j=1}^t\omega(f_j) \\ & = \sum_{i=1}^nl(y_i,(\hat{y}^{(t-1)}_i+f_t(x_i)))+\omega(f_t)+\textit{constant} \end{alignat}$

By $\textit{Taylor Series}$ expansion, \
$\begin{alignat}{1}l(y_i,(\hat{y}^{(t)}_i+f_{(t+1)}(x_i)))&=l(y_i,\hat{y}^{(t)})+\Big[\frac{\delta}{\delta\hat{y}^{(t)}}l(y_i,\hat{y}_i^{(t)})\Big]f_{(t+1)}(x_i)\nonumber\\&+\frac{1}{2}\Big[\frac{\delta^2}{\delta(\hat{y}^{(t)})^{2}}l(y_i,\hat{y}_i^{(t)})\Big]f_{(t+1)}^2(x_i)\end{alignat}$

$\begin{alignat}{1}\implies\textit{obj}^{(t)}=\sum_{i=1}^n\Big[l(y_i,\hat{y}_i^{(t-1)})+g_if_t(x_i)+\frac{1}{2}h_if_t^2(x_i)\Big]+\omega(f_t)+\textit{constant}\end{alignat}$, where \
$g_i=\frac{\delta}{\delta\hat{y}^{(t-1)}}l(y_i,\hat{y}_i^{(t-1)})$ \
$h_i=\frac{\delta^2}{\delta(\hat{y}^{(t-1)})^{2}}l(y_i,\hat{y}_i^{(t-1)})$

After we remove all the constants, the specific objective at step $t$ becomes \
$\begin{alignat}{1}{obj}^{(t)}=\sum_{i=1}^n\Big[g_if_t(x_i)+\frac{1}{2}h_if_t^2(x_i)\Big]+\omega(f_t)\end{alignat}$

This becomes our optimization goal for the new tree. One important advantage of this definition is that the value of the objective function only depends on $g_i$ and $h_i$. This is how XGBoost supports custom loss functions. We can optimize every loss function, including logistic regression and pairwise ranking, using exactly the same solver that takes $g_i$ and $h_i$ as input!

# Model Complexity
In order to define the complexity of the tree $\omega(f)$, we first refine the definition of a tree $f(x)$. \
$f_t(x)=\omega_{q(x)}, \omega \in \mathbb{R}^T, q:\mathbb{R}^d\to \{1,2,...,T\}$ \

Here $\omega$ is the vector of scores on leaves, $q$ is the function assigning each data-point to a leaf, and $T$ is the total number of leaves.

In XGBoost, the complexity is defined as:\
$\begin{alignat}{1}\omega(f)=\gamma{T}+\frac{1}{2}\lambda\sum_{j=1}^T\omega_j^2\end{alignat}$

There is more than one way to define the complexity, but this one works well in practice. The regularization is one part most tree packages treat less carefully, or simply ignore. This is because the traditional treatment of tree learning only emphasized improving impurity, while the complexity control was left to heuristics. By defining it formally, we can get a better idea of what we are learning and obtain models that perform well in the wild.

# The Structure Score

After re-formulating the tree-model, we can write the objective value with the $t^{th}$ tree as \
$\begin{alignat}{1}obj^{(t)}&\approx\sum_{i=1}^n\Big[g_i\omega_{q(x_i)}+\frac{1}{2}h_i\omega_{q(x_i)}^2\Big]+\gamma{T}+\frac{1}{2}\lambda\sum_{j=1}^T\omega_j^2\\ &= \sum_{j=1}^T\Big[ \big(\sum_{i \in I_j} g_i \big)\omega_j+\frac{1}{2}\big(\sum_{i \in I_j} h_i + \lambda\big)\omega_j^2 \Big]+\gamma{T}\\ &= \sum_{j=1}^T\Big[ G_j\omega_j+\frac{1}{2}(H_j+\lambda)\omega_j^2 \Big]+\gamma{T}\end{alignat}$ \
where, \
$I_j=\{i|q(x_i)=j\}$ is the set of indices of data-points assigned to leaf $j$\
$G_j=\sum_{i\in I_j}g_i$, and \
$H_j=\sum_{i\in I_j}h_i$

It should be noted that each $\omega_j$ is independent, and that $G_j\omega_j+\frac{1}{2}(H_j+\lambda)\omega_j^2$ is a quadratic in $\omega_j$.

Hence, the best $\omega_j$ for the structure $q(x)$ and the best corresponding objective reduction that we can get are:\
$\begin{align*} \omega_j^{*} &= -\frac{G_j}{H_j+\lambda} \\ obj^{*} &= -\frac{1}{2}\sum_{j=1}^T\frac{G_j^2}{H_j+\lambda}+\gamma{T}\end{align*}$

The last equation measures how good a tree structure $q(x)$ is. The lesser the value, better the structure is.



